In [ ]:
#测试
import numpy as np
from scipy import signal

def apply_bandpass_filters(eeg_data, fs, l=4, rs=30, order=4, start_freq=4, end_freq=40):
    """
    使用多个Chebyshev II型带通滤波器处理EEG数据。
    
    参数:
    eeg_data: numpy array, 原始EEG数据，形状为 (n_epochs, n_channels, n_samples)
    fs: int, 采样率 (Hz)
    l: int, 带通滤波器步长 (Hz)
    rs: int, 阻带纹波 (dB)
    order: int, 滤波器阶数
    start_freq: int, 带通滤波器的起始频率 (Hz)
    end_freq: int, 带通滤波器的终止频率 (Hz)

    返回:
    filtered_data: numpy array, 形状为 (n_epochs, n_bands, n_channels, n_samples)
    """
    n_epochs, n_channels, n_samples = eeg_data.shape
    filtered_data = []

    # 逐步生成带通滤波器并应用
    for low in range(start_freq, end_freq, l):
        high = low + l
        if high > end_freq:
            break

        # 设计 Chebyshev II 型带通滤波器
        sos = signal.cheby2(order, rs, [low, high], btype='band', fs=fs, output='sos')

        # 对每个epoch的每个通道进行滤波
        band_filtered = np.array([signal.sosfilt(sos, epoch, axis=-1) for epoch in eeg_data])

        # 添加到滤波结果中，形状为 (n_epochs, n_channels, n_samples)
        filtered_data.append(band_filtered)
        print(f"Filtered data for band {low}-{high} Hz.")

    # 将所有频段结果组合成一个numpy数组，形状为 (n_epochs, n_bands, n_channels, n_samples)
    filtered_data = np.stack(filtered_data, axis=1)

    return filtered_data

# 加载 rest1 和 rest2 的数据
datapath1 = r'D:\JQ_YJS\飞行试验数据\处理后\rest1.npy'
datapath2 = r'D:\JQ_YJS\飞行试验数据\处理后\rest2.npy'
rest1 = np.load(datapath1)  # 形状为 (1000, n_channels, n_samples)
rest2 = np.load(datapath2)  # 形状为 (1000, n_channels, n_samples)

# 假设采样率为 fs = 256 Hz
fs = 256
l = 4  # 滤波步长 (Hz)
start_freq = 4  # 滤波起始频率 (Hz)
end_freq = 40  # 滤波终止频率 (Hz)

# 对 rest1 和 rest2 数据应用带通滤波器
filtered_rest1 = apply_bandpass_filters(rest1, fs, l=l, rs=30, order=4, start_freq=start_freq, end_freq=end_freq)
filtered_rest2 = apply_bandpass_filters(rest2, fs, l=l, rs=30, order=4, start_freq=start_freq, end_freq=end_freq)

# 保存滤波后的数据为 .npy 文件
np.save(r'D:\JQ_YJS\分频段\filtered_rest1.npy', filtered_rest1)
np.save(r'D:\JQ_YJS\分频段\filtered_rest2.npy', filtered_rest2)

# 检查滤波后的形状
print(f"Filtered rest1 shape: {filtered_rest1.shape}")  # 应该为 (1000, n_bands, n_channels, n_samples)
print(f"Filtered rest2 shape: {filtered_rest2.shape}")